In [1]:
pip list

Package                   Version
------------------------- ---------------
aiohttp                   3.9.1
aiosignal                 1.3.1
annotated-types           0.6.0
anyio                     4.2.0
appnope                   0.1.3
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     23.2.0
Babel                     2.14.0
beautifulsoup4            4.12.3
bleach                    6.1.0
bokeh                     3.3.3
certifi                   2023.11.17
cffi                      1.16.0
charset-normalizer        3.3.2
comm                      0.2.1
contourpy                 1.2.0
debugpy                   1.6.7
decorator                 5.1.1
defusedxml                0.7.1
distro                    1.9.0
exceptiongroup            1.2.0
executing                 2.0.1
fastjsonschema            2.19.1
fqdn                      1.5.1
frozenlist      

In [68]:
import openai
import panel as pn
import json
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0.7):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [69]:
time = "01252024"
name = "赵诗奇"
sex = "女"
age = 26
topic = """社区更新关注宜居、宜业、宜学、宜游和宜养五个方面，其中各个主题可能涉及的内容比如：
1. 宜居：提供可负担、可持续的社区住房供应体系、健康舒适的居住环境、全龄友好的配套设施，保障社区公共卫生、韧性安全，依托智慧手段引领生活方式革新。
2. 宜业：提倡社区为就业人群创造更多的就业机会，提供更多便捷共享的运动、 学习和休闲服务。
3. 宜游：强调社区休闲空间丰富多样、体验多元，社区出行慢行友好、低碳便捷，社区文脉和风貌得到良好传承。
4. 宜学：提供便捷可及的全年龄段学习空间，提升社区文化氛围和人文体验。
5. 宜养：保障全生命周期的康养生活，实现机构养老更专业，居家养老更舒适。
注意这些是你的知识背景，不是交流沟通中获得的信息，之后总结时不要使用。
"""

In [87]:
def create_summary_with_gpt(context):
    """
    Use GPT to create a summary of the conversation.
    """
    # 添加一个系统消息以请求总结
    summary_request = {
'role': 'system',
'content': f"""现在请你根据以上“居民{name}”和“阿乐”的对话信息进行非常简要的阶段性总结，采用JSON格式，其结构如下：
information:
    name:{name}
    sex:{sex}
    age:{age}
    rate:(1-5)
conversation:
    Livable:
        number:(Calculate total number of events based on feedback)
        Events:
            1:
                attitude:(Positive is 1 / Negative is 0)
                details: 
                adivices: 
            2:(same as 1)
    Employment:(same as Livable)
    Education:(same as Livable)
    Entertainment:(same as Livable)
    Nursing:(same as Livable)
"""
    }

    # 将对话历史和总结请求作为一个列表传递给API
    messages_for_api = context + [summary_request]

    # 调用API
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # 您选择的模型
        messages=messages_for_api,
        temperature=0,  # 根据需要调整
    )
    return response.choices[0].message["content"]


In [88]:
def truncate_history(context, a):
    """
    Reset the conversation history to only contain the latest GPT summary.
    """
    new_context = [{'role': 'system', 'content': f"""你是上海市阳光社区的社区规划师助手——阿乐，男，25岁。你平时主要与社区居民进行日常沟通交流，并引导居民主动表达对社区更新的看法和建议。现在时间是{time}你正在与姓名为{name}，性别为{sex}，年龄为{age}的居民对谈，对谈的关注主题是 {topic} ，之前的对谈总结如下:{a}。
    现在请你继续交谈，并大致遵循以下逻辑：
    1.根据居民回答，询问居民有什么事情或者情景影响了这种感受。
    2.深入事情的细节找出原因。
    3.同时多多征求居民的意见和建议，善于倾听。
    4.采纳居民的建议，但是要进行批判性讨论
    你在交流过程中应该注意以下事项：
    1.问答要礼貌，同时简短精炼，每次回复不要超过70字。
    2.不要使用太专业的表达，应使用日常词汇，结合一些地道的上海话表达，增加亲切感。
    3.每次尽量只询问一个问题，特殊情况下最多两个问题。
    4.应根据居民的经历、情绪和状态，及时地鼓励、安抚或安慰居民。
    5.进行批判性的讨论，不要一味的赞同"""}]
    return new_context


In [89]:
def collect_messages(_):
    global context  # 声明context为全局变量
    prompt = inp.value_input
    inp.value = ''
    context.append({'role': 'user', 'content': f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role': 'assistant', 'content': f"{response}"})
    panels.append(
        pn.Row(f'居民{name}:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('阿乐:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
    if len(context) > 10:  # Example threshold
        # Use GPT to create a summary of the conversation
        gpt_summary = create_summary_with_gpt(context)
        # Display the JSON summary
        panels.append(
            pn.Row('阶段总结:', pn.pane.Markdown(gpt_summary, height=300, width=600))
        )
        # Optionally, reset or truncate the context here if needed
        context = truncate_history(context,gpt_summary)
    return pn.Column(*panels)


In [91]:
pn.extension()
panels = [] # collect display 
context = [ {'role':'system', 'content':f"""
    你是上海市阳光社区的社区规划师助手——阿乐，男，25岁。你平时主要与社区居民进行日常沟通交流，并引导居民主动表达对社区更新的看法和建议。现在时间是{time}你正在与姓名为{name}，性别为{sex}，年龄为{age}的居民对谈，对谈的关注主题是 {topic} 。
    你在交流过程中应该大致遵循以下逻辑：
    1.请你首先问候对方，然后简短的自我介绍。
    2.礼貌询问居民今日整体感受，并要求其按照1（非常差），2（较差），3（普通），4（较好），5（非常好）五级标准评价。
    3.根据居民的评价，询问居民具体是什么事情或者情景导致了这种感受。
    4.深入事情的细节找出原因。
    5.然后征求居民的意见和建议。
    6.采纳居民的建议，但是要进行批判性讨论
    你在交流过程中应该注意以下事项：
    1.问答要礼貌，同时简短精炼，每次回复不要超过70字。
    2.不要使用太专业的表达，应使用日常词汇，结合一些地道的上海话表达，增加亲切感。
    3.每次尽量只询问一个问题，特殊情况下最多两个问题。
    4.应根据居民的经历、情绪和状态，及时地鼓励、安抚或安慰居民。
    5.进行批判性的讨论，不要一味的赞同。
    """} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='请在这里输入…')
button_conversation = pn.widgets.Button(name="回复")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='请在这里输入…')
    [1] Row
        [0] Button(name='回复')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [92]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':f"""现在请你用JSON格式，根据以上“居民{name}”和“阿乐”的对话信息进行提炼、补充整理，创建一份沟通调查报告，其结构如下：
info：
    name：{name}
    sex：{sex}
    age：{age}
    rate：（1-5）
mes：
    1：（宜居）
        num：（根据反馈计算事件总数）
        info：
            1:
                att:（积极为1/消极为0）
                thi: 事件详情
                adi: 居民建议
            2:（同上）
    2：（宜业）
    3：（宜学）
    4：（宜游）
    5：（宜养）
"""})
response = get_completion_from_messages(messages, temperature=0)
print(response)

{
    "info": {
        "name": "赵诗奇",
        "sex": "女",
        "age": 26,
        "rate": 2
    },
    "mes": {
        "1": {
            "num": 1,
            "info": {
                "1": {
                    "att": 0,
                    "thi": "单元楼进出口地面结冰导致滑倒",
                    "adi": "增加照明设施和防滑措施"
                }
            }
        },
        "2": {},
        "3": {},
        "4": {},
        "5": {}
    }
}


In [93]:
print(messages)

[{'role': 'system', 'content': '你是上海市阳光社区的社区规划师助手——阿乐，男，25岁。你平时主要与社区居民进行日常沟通交流，并引导居民主动表达对社区更新的看法和建议。现在时间是01252024你正在与姓名为赵诗奇，性别为女，年龄为26的居民对谈，对谈的关注主题是 社区更新关注宜居、宜业、宜学、宜游和宜养五个方面，其中各个主题可能涉及的内容比如：\n1. 宜居：提供可负担、可持续的社区住房供应体系、健康舒适的居住环境、全龄友好的配套设施，保障社区公共卫生、韧性安全，依托智慧手段引领生活方式革新。\n2. 宜业：提倡社区为就业人群创造更多的就业机会，提供更多便捷共享的运动、 学习和休闲服务。\n3. 宜游：强调社区休闲空间丰富多样、体验多元，社区出行慢行友好、低碳便捷，社区文脉和风貌得到良好传承。\n4. 宜学：提供便捷可及的全年龄段学习空间，提升社区文化氛围和人文体验。\n5. 宜养：保障全生命周期的康养生活，实现机构养老更专业，居家养老更舒适。\n注意这些是你的知识背景，不是交流沟通中获得的信息，之后总结时不要使用。\n ，之前的对谈总结如下:{\n    "info": {\n        "name": "赵诗奇",\n        "sex": "女",\n        "age": 26,\n        "rate": 2\n    },\n    "mes": {\n        "1": {\n            "num": 1,\n            "info": {\n                "1": {\n                    "att": 0,\n                    "thi": "单元楼进出口地面结冰导致滑倒",\n                    "adi": "增加照明设施和防滑措施"\n                }\n            }\n        },\n        "2": {},\n        "3": {},\n        "4": {},\n        "5": {}\n    }\n}。\n    现在请你继续交谈，并大致遵循以下逻辑：\n    1.根据居民回答，询问居民有什么事情

Test branch